In [1]:
from pycaret.datasets import get_data

import pickle
def load_pkl(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
from pycaret.regression import *
import tqdm
import pandas as pd

In [2]:
top5list = {}
random_state=42
for name in tqdm.tqdm([4,5,8]):
    name= str(name)
    juice = pd.read_csv(f'./data{name}.csv')
    exp = RegressionExperiment()
    train_df = juice.head(100)
    test_df = juice.tail(60)
    exp.set_config('seed', random_state)
    exp_name = exp.setup(data = train_df,  target = 'label',verbose=False)
    top5 = exp.compare_models(sort = 'RMSE',n_select = 5,budget_time=360,verbose=True,include=list(exp.models().reset_index()['ID']))
    top5list[name] = [top5,exp._display_container[1],exp ] 

  0%|          | 0/3 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.1041,1.8191,1.3211,0.1507,0.4537,0.4312,0.5900
knn,K Neighbors Regressor,1.1200,1.9463,1.3625,0.0543,0.4416,0.4350,0.5530
kr,Kernel Ridge,1.1459,1.9551,1.3754,0.1054,0.4548,0.4794,0.5490
svm,Support Vector Regression,1.1795,2.0094,1.3855,0.1301,0.4538,0.5057,0.5580
ridge,Ridge Regression,1.1556,1.9813,1.3888,0.0801,0.4501,0.4997,0.6600
mlp,MLP Regressor,1.1445,1.9844,1.3988,0.0340,0.4688,0.4993,0.5730
br,Bayesian Ridge,1.1550,2.0834,1.4274,0.0168,0.4575,0.5041,0.6640
ada,AdaBoost Regressor,1.1653,2.2955,1.4349,0.0132,0.4716,0.4700,0.5610
catboost,CatBoost Regressor,1.2266,2.3233,1.4738,-0.0377,0.4905,0.4993,1.3500
lightgbm,Light Gradient Boosting Machine,1.2260,2.4327,1.4858,-0.0198,0.4796,0.5246,10.1280


 33%|███▎      | 1/3 [05:41<11:22, 341.30s/it]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,1.2465,2.4538,1.5259,-0.3066,0.5353,0.4744,0.5620
ard,Automatic Relevance Determination,1.2637,2.4282,1.5276,-0.2831,0.5253,0.4902,0.5920
huber,Huber Regressor,1.2650,2.4721,1.5285,-0.3502,0.5469,0.5186,0.5570
catboost,CatBoost Regressor,1.2835,2.4179,1.5357,-0.2769,0.5313,0.5040,1.5910
lar,Least Angle Regression,1.2548,2.5071,1.5365,-0.3409,0.5434,0.4775,0.6070
lr,Linear Regression,1.2548,2.5071,1.5365,-0.3409,0.5434,0.4775,0.5720
br,Bayesian Ridge,1.2756,2.4788,1.5507,-0.2897,0.5325,0.4818,0.6440
tr,TheilSen Regressor,1.2721,2.6746,1.5656,-0.4697,0.5470,0.4955,0.8310
rf,Random Forest Regressor,1.3224,2.6794,1.6046,-0.4613,0.5477,0.5393,0.6150
gbr,Gradient Boosting Regressor,1.3402,2.9004,1.6266,-0.5830,0.5554,0.5836,3.2510


 67%|██████▋   | 2/3 [12:41<06:27, 387.49s/it]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.0394,1.8578,1.3078,-0.0979,0.4577,0.4055,0.6210
mlp,MLP Regressor,1.1370,2.1154,1.4133,-0.6236,0.4835,0.4514,1.7730
lr,Linear Regression,1.1892,2.2518,1.4260,-0.4402,0.4935,0.5235,0.5700
knn,K Neighbors Regressor,1.1857,2.1640,1.4285,-0.7052,0.4779,0.5144,0.5760
ridge,Ridge Regression,1.1635,2.1394,1.4292,-0.5175,0.4798,0.4949,0.5560
kr,Kernel Ridge,1.1638,2.1472,1.4308,-0.5044,0.4800,0.4872,0.5850
svm,Support Vector Regression,1.1676,2.1958,1.4343,-0.5965,0.4722,0.4537,0.5760
br,Bayesian Ridge,1.1798,2.1832,1.4430,-0.5154,0.4857,0.4988,0.5950
tr,TheilSen Regressor,1.2206,2.4124,1.4686,-0.6753,0.4786,0.5346,1.0810
ard,Automatic Relevance Determination,1.2167,2.3843,1.4705,-0.4313,0.5086,0.5021,0.5590


100%|██████████| 3/3 [19:30<00:00, 390.06s/it]


In [3]:
all_result = pd.DataFrame()
for i in top5list.keys():
    t = top5list[i][1]
    t['random'] = i
    all_result=all_result.append(t)

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

def get_metric(true_labels,predicted_labels):
    # 计算准确率
    accuracy = accuracy_score(true_labels, predicted_labels)
    #print("Accuracy:", accuracy)

    # 计算精确度、召回率和F1分数
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')
    f1 = f1_score(true_labels, predicted_labels, average='micro')
    #print("Precision:", precision)
    #print("Recall:", recall)
    #print("F1 Score:", f1)

    # 计算accuracy within ±1 mRS category
    within_1_accuracy = sum(abs(a - b) <= 1 for a, b in zip(true_labels, predicted_labels)) / len(true_labels)
    #print("Accuracy within ±1 mRS category:", within_1_accuracy)

    return [accuracy,within_1_accuracy,precision,recall,f1]


In [6]:
def map_to_class(predicted_value,thouhold=.5):
    thresholds = list(range(1,7))
    thresholds = [i+thouhold for i in thresholds]
    #print(thresholds)
    for i, threshold in enumerate(thresholds):
        if predicted_value <= threshold:
            return i + 1  # 类别从1开始
    return 6  # 如果连续变量值大于5.5，则映射到类别6

In [11]:
result_all = []
for i in top5list.keys():
    exp = top5list[i][-1]
    for k in top5list[i][0]:
    
        juice = pd.read_csv(f'./data{i}.csv').head(100)

        juice['predict'] = list(exp.predict_model(k,data=juice.iloc[:,:-1]).prediction_label)
        juice['predict'] = juice['predict'].apply(map_to_class)

        result = get_metric(juice['predict'],juice['label'])
        result.append(k)
        result_all.append(result)

In [12]:
result_all

[[0.68,
  0.89,
  0.68,
  0.68,
  0.68,
  ExtraTreesRegressor(n_jobs=-1, random_state=7100)],
 [0.32, 0.73, 0.32, 0.32, 0.32, KNeighborsRegressor(n_jobs=-1)],
 [0.25, 0.67, 0.25, 0.25, 0.25, KernelRidge()],
 [0.33, 0.77, 0.33, 0.33, 0.33, SVR()],
 [0.23, 0.67, 0.23, 0.23, 0.23, Ridge(random_state=7100)],
 [0.29, 0.71, 0.29, 0.29, 0.29, Ridge(random_state=5376)],
 [0.29, 0.73, 0.29, 0.29, 0.29, ARDRegression(n_iter=1000)],
 [0.28, 0.77, 0.28, 0.28, 0.28, HuberRegressor()],
 [0.66,
  0.9,
  0.66,
  0.66,
  0.66,
 [0.3, 0.74, 0.3, 0.3, 0.3, Lars(random_state=5376)],
 [0.71,
  0.87,
  0.71,
  0.71,
  0.7100000000000001,
  ExtraTreesRegressor(n_jobs=-1, random_state=7536)],
 [0.43,
  0.83,
  0.43,
  0.43,
  0.42999999999999994,
  MLPRegressor(max_iter=500, random_state=7536)],
 [0.32, 0.8, 0.32, 0.32, 0.32, LinearRegression(n_jobs=-1)],
 [0.33, 0.72, 0.33, 0.33, 0.33, KNeighborsRegressor(n_jobs=-1)],
 [0.34, 0.74, 0.34, 0.34, 0.34, Ridge(random_state=7536)]]

In [10]:
exp.predict_model(k,data=juice.iloc[:,:-1])

AttributeError: 'list' object has no attribute 'predict'

In [9]:
top5list[i][-1]

TypeError: 'RegressionExperiment' object is not subscriptable

In [8]:
for i in top5list.keys():
    exp = top5list[i][-1]
    for k in top5list[i][0]:
        juice = pd.read_csv(f'./data{i}.csv')
        
        juice['predict'] = exp.predict_model(k,data=juice.iloc[:,:-1])
        juice['predict'] = juice['predict'].apply(map_to_class)
        
        
        name = (k)
        

TypeError: 'Lasso' object is not iterable

In [11]:
 top5list[i][0]

[]

In [9]:
exp.predict_model(k,data=juice.iloc[:,:-1])

NameError: name 'k' is not defined

In [17]:
k

Ridge(random_state=332)

In [14]:
k

Ridge(random_state=332)

In [7]:
train_df

,ED_original_shape_Sphericity,性别,HM_original_shape_LeastAxisLength,HM_ACA_R_Ratio,ED_NCCT_original_firstorder_Skewness,HM_original_shape_Elongation,ED_ACA_R_Ratio,HM_original_shape_Maximum2DDiameterColumn,镇静、镇痛治疗,ED_original_shape_Maximum2DDiameterColumn,...,发病到首次影像检查时间间隔,ED_NCCT_original_firstorder_Entropy,止吐护胃,HM_PCA_R_Ratio,ED_NCCT_original_firstorder_Uniformity,冠心病史,ED_original_shape_Maximum3DDiameter,HM_Cerebellum_R_Ratio,年龄,label
0,0.158800,0.0,0.419850,0.000521,0.168612,0.707550,0.193909,0.334207,1.0,0.545411,...,0.094737,0.296358,1.0,0.143941,0.667961,0.0,0.551518,0.000000,0.196970,4.0
1,0.243565,1.0,0.343714,1.000000,0.448773,0.605696,0.352995,0.309257,0.0,0.525105,...,0.115789,0.478909,1.0,0.362184,0.438272,0.0,0.461163,0.000000,0.424242,0.0
2,0.130834,1.0,0.503207,0.108302,0.515494,0.492735,0.289588,0.441673,1.0,0.636252,...,0.073684,0.505907,1.0,0.119421,0.400998,0.0,0.684317,0.000000,0.727273,5.0
3,0.266481,1.0,0.484321,0.004520,0.381890,0.215563,0.037174,0.356396,0.0,0.429172,...,0.031579,0.338016,0.0,0.034863,0.562145,0.0,0.541047,0.000000,0.606061,4.0
4,0.094947,1.0,0.210188,0.000000,0.420716,0.933752,0.079288,0.142206,0.0,0.973369,...,0.200000,0.320019,1.0,0.001373,0.573329,0.0,0.939951,0.000000,0.318182,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.365042,0.0,0.131598,0.000000,0.072513,0.882800,0.000000,0.293907,1.0,0.306196,...,0.031579,0.286552,1.0,0.000000,0.660860,0.0,0.275472,0.000000,0.954545,4.0
96,0.415727,0.0,0.120782,0.000000,0.520966,0.584718,0.000000,0.076243,1.0,0.292794,...,0.115789,0.576433,1.0,0.010865,0.340091,0.0,0.280565,0.000000,0.803030,2.0
97,0.077018,1.0,0.733362,0.356022,0.788004,0.268923,0.787399,0.516994,1.0,0.721933,...,0.094737,0.209613,1.0,0.180863,0.763853,1.0,0.980298,0.000000,0.787879,5.0
98,0.169748,1.0,0.341595,0.001891,0.370105,0.370642,0.013659,0.263548,1.0,0.406127,...,0.115789,0.267301,1.0,0.025785,0.694218,0.0,0.590700,0.000000,0.348485,3.0


In [4]:
all_result.groupby(['random']).head(10)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),random
ridge,Ridge Regression,1.3085,2.5617,1.5411,-0.0647,0.5140,0.5317,0.597,0
kr,Kernel Ridge,1.3041,2.5774,1.5449,-0.0773,0.5173,0.5182,0.514,0
ard,Automatic Relevance Determination,1.3135,2.5926,1.5498,-0.0839,0.5146,0.5333,0.590,0
br,Bayesian Ridge,1.3231,2.6142,1.5598,-0.0853,0.5186,0.5398,0.581,0
svm,Support Vector Regression,1.2951,2.6208,1.5606,-0.0798,0.5210,0.5101,0.549,0
...,...,...,...,...,...,...,...,...,...
mlp,MLP Regressor,1.3445,2.9879,1.6481,-0.1306,0.5393,0.5841,3.325,8
br,Bayesian Ridge,1.4017,3.0206,1.6866,-0.1382,0.5453,0.5547,0.532,8
et,Extra Trees Regressor,1.3966,3.1587,1.7006,-0.1804,0.5514,0.5379,3.194,8
lasso,Lasso Regression,1.4730,3.0318,1.7123,-0.1318,0.5527,0.5797,0.537,8
